## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:侯益康


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

### 本次作业基于大模型辅助完成的作业题已在idea处或者code处进行声明

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：
![](fig/3-1.jpg)

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：
![](fig/3-2.jpg)

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：
1. 排序: 首先，将所有课程按照结束时间从早到晚进行排序。如果结束时间相同，可以按照开始时间从早到晚排序（这一步对于本问题的核心逻辑不是绝对必要，但有助于保持一致性）。
2. 分配教室:
- 初始化一个空的教室列表，每个教室也维护一个它已经安排的课程列表。
- 遍历排序后的课程列表：
  - 对于当前课程，尝试将其分配给一个已有的、并且该课程的开始时间晚于或等于该教室中最后一门课程结束时间的教室。
  - 如果在所有现有教室中都找不到合适的（即都会发生时间冲突），则开设一个新的教室，并将当前课程分配给这个新教室。
  - 为了优化查找过程，我们可以优先选择那个最后一节课结束时间最早，并且又能容纳当前课程的教室。但这在本题的“最少教室”目标下，只要找到一个能放的就行。更简单的方法是：遍历所有教室，如果当前课程可以放入某个教室（即当前课程的开始时间晚于或等于该教室最后一节课的结束时间），就放入，并更新该教室的最后一节课结束时间。
3. 记录结果 (Record Results):
- 最终，教室列表中的教室数量就是所需的最小教室数量。
- 每个教室中记录的课程就是该教室的课程安排。

In [1]:
#code：代码为大模型生成
def schedule_courses(courses_str):
    
    if not courses_str:
        return 0, []

    # 1. 解析和转换时间
    parsed_courses = []
    for course_s in courses_str:
        start_str, end_str = course_s.split(',')
        start_h, start_m = map(int, start_str.split(':'))
        end_h, end_m = map(int, end_str.split(':'))
        start_minutes = start_h * 60 + start_m
        end_minutes = end_h * 60 + end_m
        parsed_courses.append(((start_h, start_m), (end_h, end_m), start_minutes, end_minutes, course_s))

    # 2. 按课程结束时间排序
    # 如果结束时间相同，按开始时间排序（可选，但有助于确定性）
    parsed_courses.sort(key=lambda x: (x[3], x[2]))

    room_schedules = [] 
    room_finish_times = [] 

    for course in parsed_courses:
        start_time_minutes = course[2]
        end_time_minutes = course[3]
        original_course_str = course[4]

        assigned = False

        best_room_idx = -1
        min_gap = float('inf') # 用于找到结束时间最接近且能容纳的教室（可选优化）

        for i in range(len(room_finish_times)):
            if room_finish_times[i] <= start_time_minutes:
                if best_room_idx == -1 or room_finish_times[i] > room_finish_times[best_room_idx] : # 优先选结束晚的，这样可以填充空隙
                     best_room_idx = i


        if best_room_idx != -1: # 找到了一个可以容纳的现有教室
            room_schedules[best_room_idx].append(original_course_str)
            room_finish_times[best_room_idx] = end_time_minutes
            # 对该教室的课程按开始时间排序（可选，为了输出美观）
            room_schedules[best_room_idx].sort(key=lambda c_str: int(c_str.split(',')[0].split(':')[0])*60 + int(c_str.split(',')[0].split(':')[1]))

        else:
            # 没有合适的现有教室，开设新教室
            room_schedules.append([original_course_str])
            room_finish_times.append(end_time_minutes)

    return len(room_schedules), room_schedules

# 根据图片中的课程数据
courses_data_str = [
    "9:00,12:30", "11:00,14:00", "13:00,14:30", "9:00,10:30",
    "13:00,14:30", "14:00,16:30", "15:00,16:30", "15:00,16:30", # 图片中有两个15:00,16:30
    "9:00,10:30"
]

min_rooms_needed, schedules = schedule_courses(courses_data_str)

print(f"最少需要的教室数量: {min_rooms_needed}")
print("每个教室的课程安排:")
for i, schedule in enumerate(schedules):
    print(f"教室 {i+1}: {schedule}")

#时间复杂度：时间复杂度是 O(N^2logN)

最少需要的教室数量: 3
每个教室的课程安排:
教室 1: ['9:00,10:30', '11:00,14:00', '14:00,16:30']
教室 2: ['9:00,10:30', '13:00,14:30', '15:00,16:30']
教室 3: ['9:00,12:30', '13:00,14:30', '15:00,16:30']


## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：要爬到第n阶楼梯，最后一步可能是从第(n-1)阶爬1阶到达，或从第(n-2)阶爬2阶到达
因此爬到第n阶的方法数 = 爬到第(n-1)阶的方法数 + 爬到第(n-2)阶的方法数
递推关系：定义dp[i]表示爬到第i阶的方法数
则有：dp[i] = dp[i-1] + dp[i-2]
边界条件：dp[1] = 1（爬1阶有1种方法），dp[2] = 2（爬2阶有2种方法：一次爬1阶两次，或一次爬2阶）
实现方式：可以使用数组存储所有状态，也可以使用滚动数组（只保存最近的两个状态）来优化空间复杂度

In [38]:
# add your code here
def climb(n):
    if n <= 0:
        return 0
    if n == 1:
        return 1
    if n == 2:
        return 2
    dp = [0] * (n + 1)
    dp[1] = 1
    dp[2] = 2
    for i in range(3, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
    
    return dp[n]

test_cases = [
    (1, 1),  
    (2, 2),  
    (3, 3), 
    (4, 5),  
    (5, 8),  
    (0, 0),  
]

all_passed = True
for n, expected in test_cases:
    result = climb(n)
    if result == expected:
        print(f"pass: 爬{n}阶楼梯，期望{expected}种方法，实际得到{result}种方法")
    else:
        print(f"error: 爬{n}阶楼梯，期望{expected}种方法，实际得到{result}种方法")
        all_passed = False

if all_passed:
    print("AC!")
else:
    print("Error!")

# your algorithm time complexity is: O(n)

pass: 爬1阶楼梯，期望1种方法，实际得到1种方法
pass: 爬2阶楼梯，期望2种方法，实际得到2种方法
pass: 爬3阶楼梯，期望3种方法，实际得到3种方法
pass: 爬4阶楼梯，期望5种方法，实际得到5种方法
pass: 爬5阶楼梯，期望8种方法，实际得到8种方法
pass: 爬0阶楼梯，期望0种方法，实际得到0种方法
AC!


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：定义状态：设 dp[i][j] 表示从前 i 个物品中选择，总重量不超过 j 的情况下能获得的最大价值。
状态转移方程：如果不选择第 i 个物品：dp[i][j] = dp[i-1][j]
如果选择第 i 个物品：dp[i][j] = dp[i-1][j-w[i]] + v[i]，其中 w[i] 和 v[i] 分别是第 i 个物品的重量和价值。
最终结果：dp[n][W]，其中 n 是物品数量，W 是背包容量。


In [39]:
# add your code here
def bag(items, capacity):
    n = len(items)
    # 创建二维dp数组，初始化为0
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    # 填充dp数组
    for i in range(1, n + 1):
        weight, value = items[i-1]
        for j in range(capacity + 1):
            if weight > j:  # 当前物品太重，放不进背包
                dp[i][j] = dp[i-1][j]
            else:  # 可以放进背包，选择放或不放中价值较大的情况
                dp[i][j] = max(dp[i-1][j], dp[i-1][j-weight] + value)
    
    # 返回最大价值
    return dp[n][capacity]

items = [(5, 10), (4, 40), (6, 30), (3, 50)]
capacity = 9
max_value = bag(items, capacity)
print(f"能够装入小推车的大理石的最大价值为: {max_value}")
# your algorithm time complexity is:O(nW)  W为背包容量

能够装入小推车的大理石的最大价值为: 90


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
1. 使用左右两个指针从两端向中间移动
2. 记录左右两边的最大高度
3. 较小的那一侧可以确定当前位置的蓄水量


In [6]:
# add your code here
def trap(height):
    if not height or len(height) < 3:
        return 0
    
    n = len(height)
    
    # 计算每个位置左侧的最大高度
    left_max = [0] * n
    left_max[0] = height[0]
    for i in range(1, n):
        left_max[i] = max(left_max[i-1], height[i])
    
    # 计算每个位置右侧的最大高度
    right_max = [0] * n
    right_max[n-1] = height[n-1]
    for i in range(n-2, -1, -1):
        right_max[i] = max(right_max[i+1], height[i])
    
    # 计算每个位置能接的雨水量
    water = 0
    for i in range(n):
        water += min(left_max[i], right_max[i]) - height[i]
    
    return water

height1 = [0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]
print(f"case1: {trap(height1)}")  

height2 = [4, 2, 0, 3, 2, 5]
print(f"case2: {trap(height2)}") 
# your algorithm time complexity is: O(n)

case1: 6
case2: 9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
1. 根据收益率对股票进行排序（从高到低）
2. 优先投资收益率最高的股票，直到预算用完或无法再投资任何一只股票
3. 计算最终的总投资和总收益

In [40]:
# add your code here
def stock(stocks, budget):
    
    sorted_stocks = sorted(stocks, key=lambda x: x['rate'], reverse=True)
    selected_stocks = []
    remaining_budget = budget
    total_profit = 0
    
    for stock in sorted_stocks:
        if stock['cost'] <= remaining_budget:
            # 如果预算足够，就投资这只股票
            selected_stocks.append(stock)
            remaining_budget -= stock['cost']
            # 计算这只股票的收益
            profit = stock['cost'] * stock['rate'] / 100
            total_profit += profit
    total_investment = budget - remaining_budget
    
    return selected_stocks, total_investment, total_profit

stocks = [
    {'name': 'A', 'cost': 5000, 'rate': 10},
    {'name': 'B', 'cost': 3000, 'rate': 8},
    {'name': 'C', 'cost': 2000, 'rate': 12}
]

budget = 9999

selected, investment, profit = stock(stocks, budget)

print("已选择的股票:")
for stock in selected:
    print(f"股票 {stock['name']}: 投资 ${stock['cost']}, 收益率 {stock['rate']}%")

print(f"\n总投资: ${investment}")
print(f"总预期收益: ${profit:.2f}")
print(f"平均收益率: {(profit/investment*100):.2f}%")
# your algorithm time complexity is:O(nlogn)

已选择的股票:
股票 C: 投资 $2000, 收益率 12%
股票 A: 投资 $5000, 收益率 10%

总投资: $7000
总预期收益: $740.00
平均收益率: 10.57%


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：
1. 对二叉树进行中序遍历，同时记录出现逆序的节点
2. 找到需要交换的两个节点
3. 交换这两个节点的值，恢复二叉搜索树的特性


In [ ]:
# add your code here
import collections
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode({self.val})"

class Solution:
    def recoverTree(self, root: TreeNode) -> None:
        nodes_list = []

        def inorder_traversal(node):
            if node is None:
                return
            inorder_traversal(node.left)
            nodes_list.append(node)
            inorder_traversal(node.right)

        inorder_traversal(root)

        first_node = None
        second_node = None
        if not nodes_list:
            return
        prev_node = nodes_list[0]
        if len(nodes_list) == 1:
             return

        for i in range(1, len(nodes_list)):
            current_node = nodes_list[i]
            if current_node.val < prev_node.val:
                second_node = current_node
                if first_node is None:
                    first_node = prev_node
                else:
                    break
            prev_node = current_node

        if first_node and second_node:
            first_node.val, second_node.val = second_node.val, first_node.val
def build_tree(nodes_val):
    if not nodes_val:
        return None
    nodes = [(TreeNode(val) if val is not None else None) for val in nodes_val]
    if not nodes or nodes[0] is None:
        return None
    root = nodes[0]
    queue = collections.deque([root])
    i = 1
    while queue and i < len(nodes):
        node = queue.popleft()
        if node: 
            if i < len(nodes) and nodes[i] is not None:
                node.left = nodes[i]
                queue.append(node.left)
            i += 1
            if i < len(nodes) and nodes[i] is not None:
                node.right = nodes[i]
                queue.append(node.right)
            i += 1
    return root
def tree_to_list(root):
    if not root:
        return []
    output = []
    queue = collections.deque([root])
    while queue:
        node = queue.popleft()
        if node:
            output.append(node.val)
            queue.append(node.left)
            queue.append(node.right)
        else:
            output.append(None)
    while output and output[-1] is None:
        output.pop()
    return output
def get_inorder_values(root):
    res = []
    def inorder(node):
        if not node:
            return
        inorder(node.left)
        res.append(node.val)
        inorder(node.right)
    inorder(root)
    return res

solver = Solution()

input_list1 = [1, 3, None, None, 2]
root1 = build_tree(input_list1)
print(f"输入列表: {input_list1}")
print(f"原始树 (中序): {get_inorder_values(root1)}")
solver.recoverTree(root1)
output_list1 = tree_to_list(root1)
print(f"恢复后树 (中序): {get_inorder_values(root1)}")
print(f"输出列表: {output_list1}") # Expected: [3, 1, null, null, 2]
print("------" * 10)
input_list2 = [3, 1, 4, None, None, 2]
root2 = build_tree(input_list2)
print(f"输入列表: {input_list2}")
print(f"原始树 (中序): {get_inorder_values(root2)}")
solver.recoverTree(root2)
output_list2 = tree_to_list(root2)
print(f"恢复后树 (中序): {get_inorder_values(root2)}")
print(f"输出列表: {output_list2}") # Expected: [2, 1, 4, null, null, 3]
# your algorithm time complexity is: O(N)

输入列表: [1, 3, None, None, 2]
原始树 (中序): [3, 2, 1]
恢复后树 (中序): [1, 2, 3]
输出列表: [3, 1, None, None, 2]
------------------------------------------------------------
输入列表: [3, 1, 4, None, None, 2]
原始树 (中序): [1, 3, 2, 4]
恢复后树 (中序): [1, 2, 3, 4]
输出列表: [2, 1, 4, None, None, 3]


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：
使用深度优先搜索遍历树。一个辅助函数 dfs(node, current_path_value) 会被调用，其中 node 是当前节点，current_path_value 是从根到 node 父节点路径所代表的数字。
1. 如果当前节点为空 ，说明此路径无效或已结束，返回 0。
2. 计算到达当前节点时形成的数字: new_value = current_path_value * 10 + node.val。
3. 如果当前节点是叶子节点 ，则 new_value 就是一个完整的路径数字，返回它。
4. 如果当前节点不是叶子节点，则递归地计算左子树和右子树的路径数字之和

In [ ]:
# add your code here
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def sumNumbers(self, root: TreeNode) -> int:
        
        def dfs(node: TreeNode, current_path_value: int) -> int:
            if not node:
                return 0

            new_value = current_path_value * 10 + node.val

            if not node.left and not node.right: 
                return new_value
            sum_left = dfs(node.left, new_value)
            sum_right = dfs(node.right, new_value)
            
            return sum_left + sum_right

        if not root:
            return 0
        return dfs(root, 0)
solver=Solution()
root1 = TreeNode(1)
root1.left = TreeNode(2)
root1.right = TreeNode(3)
result1 = solver.sumNumbers(root1)
print(f"case 1:")
print(f"  树结构: [1,2,3]") 
print(f"  实际输出: {result1}")

root2 = TreeNode(4)
root2.left = TreeNode(9)
root2.right = TreeNode(0)
root2.left.left = TreeNode(5)
root2.left.right = TreeNode(1)
result2 = solver.sumNumbers(root2)
print(f"case 2:")
print(f"  树结构: [4,9,0,null,null,5,1]")
print(f"  实际输出: {result2}")
# your algorithm time complexity is: O(N)

case 1:
  树结构: [1,2,3]
  实际输出: 25
case 2:
  树结构: [4,9,0,null,null,5,1]
  实际输出: 1026


## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea: 
1. 递归
- 它们的根节点值相同
- 第一棵树的左子树和第二棵树的右子树互为镜像
- 第一棵树的右子树和第二棵树的左子树互为镜像
- 时间复杂度为O(n)

2. 迭代
首先将根节点的左右子树根节点分别入队，然后每次从队列中取出两个节点，比较它们是否相同。如果相同，则将它们的子节点以镜像的顺序插入队列中，时间复杂度为O(n)


In [2]:
# add your code here
from collections import deque
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def is_symmetric(root):
    if not root:
        return True
    def is_mirror(left, right):
        if not left and not right:
            return True
        if not left or not right:
            return False
        if left.val != right.val:
            return False
        return is_mirror(left.left, right.right) and is_mirror(left.right, right.left)
    
    return is_mirror(root.left, root.right)

def is_symmetric_iterative(root):
    if not root:
        return True
    
    queue = deque([(root.left, root.right)])
    
    while queue:
        left, right = queue.popleft()
        if not left and not right:
            continue
        if not left or not right or left.val != right.val:
            return False
        queue.append((left.left, right.right))
        queue.append((left.right, right.left))
    return True

def build_tree_from_list(lst):
    if not lst:
        return None
    
    nodes = []
    for val in lst:
        if val is None or val == 'null':
            nodes.append(None)
        else:
            nodes.append(TreeNode(val))
    
    for i in range(len(nodes)):
        if nodes[i]:
            left_idx = 2 * i + 1
            right_idx = 2 * i + 2
            if left_idx < len(nodes):
                nodes[i].left = nodes[left_idx]
            if right_idx < len(nodes):
                nodes[i].right = nodes[right_idx]
    
    return nodes[0]

a = [1, 2, 2, 3, 4, 4, 3]
b = [1, 2, 2, None, 3, None, 3]
print("---递归---")
root1 = build_tree_from_list(a)
result1 = is_symmetric(root1)
print(f"输入: root = {a}, 输出: {result1}")

root2 = build_tree_from_list(b)
result2 = is_symmetric(root2)
print(f"输入: root = {b}, 输出: {result2}")
print("---迭代---")
root1 = build_tree_from_list(a)
result1 = is_symmetric_iterative(root1)
print(f"输入: root = {a}, 输出: {result1}")
root2 = build_tree_from_list(b)
result2 = is_symmetric_iterative(root2)
print(f"输入: root = {b}, 输出: {result2}")

# your algorithm time complexity is: O(N)

---递归---
输入: root = [1, 2, 2, 3, 4, 4, 3], 输出: True
输入: root = [1, 2, 2, None, 3, None, 3], 输出: False
---迭代---
输入: root = [1, 2, 2, 3, 4, 4, 3], 输出: True
输入: root = [1, 2, 2, None, 3, None, 3], 输出: False


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:
1. 遍历整个网格，当发现一个'1'时，从这个点开始进行DFS或BFS
2. 在搜索过程中，将所有相连的'1'标记为已访问
3. 每次开始一个新的搜索，岛屿数量加1


In [21]:
# add your code here
def numIslands(grid):
    if not grid:
        return 0
    
    rows = len(grid)
    cols = len(grid[0])
    count = 0
    
    def dfs(r, c):
        # 检查边界条件和是否为陆地
        if (r < 0 or r >= rows or c < 0 or c >= cols or grid[r][c] == '0'):
            return
        # 标记当前单元格为已访问
        grid[r][c] = '0'
        # 递归搜索四个方向的相邻单元格
        dfs(r+1, c)  # 下
        dfs(r-1, c)  # 上
        dfs(r, c+1)  # 右
        dfs(r, c-1)  # 左
    # 遍历网格
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                count += 1
                dfs(r, c)
    return count
grid1 = [
    ["1","1","1","1","0"],
    ["1","1","0","1","0"],
    ["1","1","0","0","0"],
    ["0","0","0","0","0"]
]
print("case1:", numIslands(grid1)) 

grid2 = [
    ["1","1","0","0","0"],
    ["1","1","0","0","0"],
    ["0","0","1","0","0"],
    ["0","0","0","1","1"]
]
print("case2:", numIslands(grid2)) 

# your algorithm time complexity is:O(m*n)

case1: 1
case2: 3
